# Проект: Анализ NPS
Заказчик данного исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:

9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Цель исследования — ответить на следующие вопросы:

Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?

Какие группы пользователей наиболее лояльны к сервису? Какие менее?

Какой общий NPS среди всех опрошенных?

Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

В таблице 14 столбцов:
    
- user_id: Идентификатор клиента
- lt_day: Количество дней «жизни» клиента
- is_new: Поле хранит информацию о том, является ли клиент новым
- age: Возраст
- gender_segment: Пол (для удобства работы с полем преобразуйте значения в текстовый вид)
- os_name: Тип операционной системы
- cpe_type_name: Тип устройства
- country: Страна проживания
- city: Город проживания
- age_segment: Возрастной сегмент
- traffic_segment: Сегмент по объёму потребляемого трафика
- lifetime_segment: Сегмент по количеству дней «жизни»
- nps_score: Оценка клиента в NPS-опросе
- nps_group: Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'Новый'
           ELSE 'Старый'
           END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment == 1 THEN 'Женщина'
           WHEN u.gender_segment == 0 THEN 'Мужчина'
           ELSE 'не указано'
           END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTR(a.title, 4) AS age_segment,
       SUBSTR(t.title, 4) AS traffic_segment,
       SUBSTR(ls.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'Cторонник'
           WHEN u.nps_score >= 7 THEN 'Нейтрал'
           ELSE 'Критик'
           END AS nps_group
FROM user AS u
JOIN location AS l ON u.location_id = l.location_id
JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(15)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Cторонник
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Cторонник
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Cторонник
3,A004TB,4190,Старый,44.0,Женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,Cторонник
4,A004XT,1163,Старый,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,Cторонник
5,A005O0,5501,Старый,42.0,Женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,Критик
6,A0061R,1236,Старый,45.0,Мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,Cторонник
7,A009KS,313,Новый,35.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,Cторонник
8,A00AES,3238,Старый,36.0,Женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,Cторонник
9,A00F70,4479,Старый,54.0,Женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,Cторонник


In [5]:
df.describe()

,lt_day,age,nps_score
count,502493.000000,501939.000000,502493.000000
mean,1868.841439,39.621946,7.508562
std,1683.701762,11.188249,3.020378
min,-21.000000,10.000000,1.000000
25%,533.000000,31.000000,5.000000
50%,1239.000000,38.000000,9.000000
75%,3064.000000,47.000000,10.000000
max,9162.000000,89.000000,10.000000


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

День добрый, Сборный проект: Анализ NPS: <https://public.tableau.com/views/sbor2_17216504081970/Dashboard3?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link>